This notebook pulls 2010 census data and attempts to generate a large set of points that approximates a smooth surface while propogating error

In [ ]:
# Declare static variables

n=5 # The number of points to assign to each census block
BINOMIAL_TRIALS = 40 # The number of trials in the binomial distribution used for weighting points in blocks. The higher the value, the more evenly distributed the population points will be through the census block
BINOMIAL_SUCCESS = 0.5 # The probability of success for each trial in he weight assignment. Must be <=1. Use 1 for a uniform distribution

# Potential other methods for weight generation:
# Apply a transformation to make the outcome more normal
# Apply a normal distribution and use min/max normalization
# Weighted Poisson binomial distribution


# Binomial distribution is nearly normal if np(1-p) >= 10
print(BINOMIAL_TRIALS * BINOMIAL_SUCCESS * (1 - BINOMIAL_SUCCESS) >= 10)

In [ ]:
# Import libraries

import pandas as pd
import geopandas
import numpy as np
import requests
from io import BytesIO
import folium
from IPython.display import clear_output
from itertools import chain

In [ ]:
# Request shapefile data for 2010 census tracts and convert to geopandas dataframe

# Shapefile url
data_url = 'https://www2.census.gov/geo/tiger/GENZ2010/gz_2010_10_140_00_500k.zip'


# Request data
data = requests.get(data_url)
# convert to pandas dataframe
tract_data = geopandas.read_file(BytesIO(data.content))

In [ ]:
# Request shapefile data for 2010 census blocks and convert to geopandas dataframe

# Shapefile url
data_url = 'https://www2.census.gov/geo/tiger/TIGER2010/TABBLOCK/2010/tl_2010_10_tabblock10.zip'


# Request data
data = requests.get(data_url)
# convert to pandas dataframe
block_data = geopandas.read_file(BytesIO(data.content))

In [ ]:
# For each census block, create a bounding box
block_bounds = block_data["geometry"].bounds

# Attch GEOID to boundaries
block_bounds = block_data[["GEOID10","geometry"]].merge(block_bounds, left_index=True, right_index=True)

In [ ]:
# Fit a 2D Binomial distribution over the bounding boxes

# Takes in a row of 'block_bounds' and outputs a 2D Gaussian distribution of 'n' points over the bounding box, as well as the GEOID
def get_points(row,n):
    print(f"Processing Block {row['GEOID10']}...")
    # 'i' is the total number of points left to assign
    i=n
    # 'points_return' is the list of all points for the block
    # TODO: CRS is hardcoded
    points_return = geopandas.GeoSeries(crs="EPSG:4269")
    # Allocate points until n have been assigned
    while i > 0:
        # Generates a uniform distribution for the y-axis located at the center of the box
        pointsy = np.random.uniform(low=row["miny"], high=row["maxy"], size=i)
        # Generates a uniform distribution for the x-axis located at the center of the box
        pointsx = np.random.uniform(low=row["minx"], high=row["maxx"], size=i)
        # Convert the points to Shapely points
        points = geopandas.GeoSeries(geopandas.points_from_xy(pointsx, pointsy, crs="EPSG:4269"))
        # Check if the points are inside the block
        point_checks = points.within(row["geometry"])
        # Add found points to our list
        points_return = geopandas.GeoSeries(pd.concat([points_return, points[point_checks]], ignore_index=True), crs=points_return.crs)
        # Set 'i' equal to the number of missed points
        i = n - points_return.size
    
    # Generates a binomial distribution of weights
    weights = np.random.binomial(n=BINOMIAL_TRIALS, p=BINOMIAL_SUCCESS, size=n)
    # Normalize weights so that they sum to 1
    weights = weights / np.sum(weights)
    # Sort the weights based on distance from the mean
    weights = weights[np.argsort(np.abs(weights - np.mean(weights)))]
    # Generate a series containing the distance from each point to the centroid
    distances = points_return.distance(row["geometry"].centroid)
    # Create a column for the index of the point and sort by distance
    distances = distances.reset_index(name="distance").sort_values(by="distance")
    # Assign a weight to each point
    distances["weight"] = weights
    # Merge weights onto points
    points_return = pd.merge(left=points_return.rename("geometry"), right=distances, how="left", right_on="index", left_index=True)[["geometry", "weight"]]
    # Clear warnings from notebook output to prevent crash
    clear_output()
    # Return an array with every point in the cloud, the weights for each point and the GEOID
    return list(chain(points_return["geometry"].values, points_return["weight"].values, [row["GEOID10"]]))
    

In [ ]:
# Fit a Gaussian distribution to each block
point_cloud = block_bounds.apply(get_points, axis=1, args=(n,), result_type='expand')

# Rename columns of the pointcloud
point_cloud.columns = ['point_' + str(x) if x<n else 'weight_' + str(x-n) if x<2*n else 'GEOID' for x in point_cloud.columns]

In [ ]:
# Pull population data for 2010 Census blocks
# Define request parameters

year = '2010' # Year of interest
datasource = 'dec' # Survey name
subsource = 'pl' # Subsurvey name
GET = 'P001001' # Variables to query
FOR = 'block:*' # for predicate
IN = 'state:10&in=county:*&in=tract:*'

# Filepath to your Census API key
keyfile = 'CensusAPIKey.txt'

# Formatted API call
data_url = f'https://api.census.gov/data/{year}/{datasource}/{subsource}?get={GET}&for={FOR}&in={IN}'

# Read Census key into 'api_key'
with open(keyfile) as key:
    api_key = key.read().strip()

# Add key to url
data_url = f'{data_url}&key={api_key}'

# Request data and convert from json
data = requests.get(data_url).json()
# First entry in list is a list of variable names
data = pd.DataFrame(data[1:], columns = data[0])

# Rename columns to match shapefile pull
data.rename(columns = {"state":"STATEFP10", "county":"COUNTYFP10", "tract":"TRACTCE10", "block":"BLOCKCE10"}, inplace=True)

# Attach to block shapes
block_data = block_data.merge(data, on=["STATEFP10","COUNTYFP10","TRACTCE10","BLOCKCE10"])

In [ ]:
# Pull population data for 2010 Census tracts
# This notebook pulls population, white population, and female population
# Define request parameters

year = '2010' # Year of interest
datasource = 'acs' # Survey name
subsource = 'acs5' # Subsurvey name
GET = 'B01003_001E,B01003_001EA,B01003_001M,B01003_001MA,B02001_002E,B02001_002EA,B02001_002M,B02001_002MA,B01001_026E,B01001_026EA,B01001_026M,B01001_026MA' # Variables to query
FOR = 'tract:*' # for predicate
IN = 'state:10' # in predicate


# Filepath to your Census API key
keyfile = 'CensusAPIKey.txt'

# Formatted API call
data_url = f'https://api.census.gov/data/{year}/{datasource}/{subsource}?get={GET}&for={FOR}&in={IN}'

# Read Census key into 'api_key'
with open(keyfile) as key:
    api_key = key.read().strip()

# Add key to url
data_url = f'{data_url}&key={api_key}'

# Request data and convert from json
data = requests.get(data_url).json()
# First entry in list is a list of variable names
data = pd.DataFrame(data[1:], columns = data[0])

# Rename columns to match shapefile pull
data.rename(columns = {"state":"STATE", "county":"COUNTY", "tract":"TRACT"}, inplace=True)

# Attach to tract shapes
tract_data = tract_data.merge(data, on=["STATE","COUNTY","TRACT"])

In [ ]:
# Check data anotations for errors
print(tract_data.loc[~tract_data["B01003_001EA"].isnull()])
print(tract_data.loc[~tract_data["B01003_001MA"].isnull()])
print(tract_data.loc[~tract_data["B02001_002EA"].isnull()])
print(tract_data.loc[~tract_data["B02001_002MA"].isnull()])
print(tract_data.loc[~tract_data["B01001_026EA"].isnull()])
print(tract_data.loc[~tract_data["B01001_026MA"].isnull()])

In [ ]:
# Look for missing tract data

tract_data.loc[pd.isna(tract_data[['B01003_001E','B01003_001M','B02001_002E','B02001_002M','B01001_026E','B01001_026M']]).any(axis=1)][['B01003_001E','B01003_001M','B02001_002E','B02001_002M','B01001_026E','B01001_026M']]


In [ ]:
# Assign a fraction of the population of each block as a value to each point

# Merge each point to the 2010 census block containing it
population_per_point = point_cloud.merge(block_data, how="left", left_on="GEOID", right_on="GEOID10")

# Multiply each weight by the block population to get the block population per point
population_per_point[[x for x in population_per_point.columns if 'weight' in x]] = population_per_point[[x for x in population_per_point.columns if 'weight' in x]].mul(population_per_point["P001001"].astype(int), axis=0)

In [ ]:
# Flatten to a GeoSeries where each row is a point and its weight
weights = np.array([[row["weight_" + str(i)] for i in range(n)] for _, row in population_per_point.iterrows()]).flatten()
points = np.array([[row["point_" + str(i)] for i in range(n)] for _, row in population_per_point.iterrows()]).flatten()
points_list = geopandas.GeoDataFrame({"population_per_point":weights,"geometry":points}, crs="EPSG:4269")


# Determine the number of points in the point cloud. This should be n * the number of census blocks
print(points_list.shape[0] / n == block_data.shape[0])

In [ ]:
# Spatially join each point to the 2010 census tract containing it
variables_per_point = geopandas.sjoin(points_list, tract_data, how="left", op='within')

In [ ]:
# Look for missing data: ~750 points with missed in the join
variables_per_point.loc[pd.isna(variables_per_point[['B01003_001E','B01003_001M','B02001_002E','B02001_002M','B01001_026E','B01001_026M',"index_right"]]).any(axis=1)][['B01003_001E','B01003_001M','B02001_002E','B02001_002M','B01001_026E','B01001_026M']]

In [ ]:
# WARNING: Plot is large and should only be rendered if necessary
# TODO: Points around the edge of the state are being lost

"""
# Find and plot all missed points 
missed_points = variables_per_point.loc[variables_per_point["index_right"].isna()]

# initialize the map and store it in a folium map object
us_map = folium.Map(location=[38.9108, -75.5277], zoom_start=8, tiles=None)

# Add background tiles
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(us_map)

# Style and highlight functions map population values to color values
style_function = lambda x: {"weight":0.5, 
                            'color':'black',
                            'fillColor':'red', 
                            'fillOpacity':0.75}

# Add a map over the tiles with the given colors and a tooltip
NIL=folium.features.GeoJson(
        missed_points, # Full geopandas data
        style_function=style_function, # function for base colors
        control=False
    )

# Add elements to map
us_map.add_child(NIL)"""

In [ ]:
# ~175 points with 0 population
variables_per_point.loc[variables_per_point["B01003_001E"].astype(float) <= 0]

In [ ]:
# Exclude missed points from the list
variables_per_point = variables_per_point.dropna(subset = ["B01003_001E", "B02001_002E", "B01001_026E","B01003_001M", "B02001_002M", "B01001_026M"])
# Exclude points corresponding to a population of 0
variables_per_point = variables_per_point.loc[variables_per_point["B01003_001E"].astype(float) > 0]


# Divide variables of intersest by tract population and multiply by the portion of the population represented by each point
variables_per_point[["B01003_001E", "B02001_002E", "B01001_026E"]] = variables_per_point[["B01003_001E", "B02001_002E", "B01001_026E"]].astype(int).div(variables_per_point["B01003_001E"].astype(int), axis=0).mul(variables_per_point["population_per_point"].astype(float), axis=0)

# Propogate Error through ratios: dg = g(a,b) sqrt([da/a]^2 + [db/b]^2)
variables_per_point["B02001_002T"] = variables_per_point["B02001_002E"].astype(int).div(variables_per_point["B01003_001E"].astype(int), axis=0).mul((variables_per_point["B02001_002M"].astype(float).div(variables_per_point["B02001_002E"].astype(float)).apply(np.square) + variables_per_point["B01003_001M"].astype(float).div(variables_per_point["B01003_001E"].astype(float)).apply(np.square)).apply(np.sqrt), axis=0)
# Propogate error for multiplication with weights: df_n = w_n * x_n * dg
variables_per_point["B02001_002T"] = variables_per_point["B02001_002T"].mul(variables_per_point["population_per_point"], axis=0)

# Repeat for other variables
# Propogate Error through ratios: dg = g(a,b) sqrt([da/a]^2 + [db/b]^2)
variables_per_point["B01001_026T"] = variables_per_point["B01001_026E"].astype(int).div(variables_per_point["B01003_001E"].astype(int), axis=0).mul((variables_per_point["B01001_026M"].astype(float).div(variables_per_point["B01001_026E"].astype(float)).apply(np.square) + variables_per_point["B01003_001M"].astype(float).div(variables_per_point["B01003_001E"].astype(float)).apply(np.square)).apply(np.sqrt), axis=0)
# Propogate error for multiplication with weights: df_n = w_n * x_n * dg
variables_per_point["B01001_026T"] = variables_per_point["B01001_026T"].mul(variables_per_point["population_per_point"], axis=0)

# For population, we do not have a way to propogate error since we are summing block data

# Reset index
variables_per_point = variables_per_point.reset_index()

In [ ]:
# Print the number of points missed in the transfer of data from tracts to points
print(points_list.shape[0] -  variables_per_point.shape[0])

In [ ]:
# Request shapefile data for 2020 census tracts and convert to geopandas dataframe

# Shapefile url
data_url = 'https://www2.census.gov/geo/tiger/GENZ2020/shp/cb_2020_10_tract_500k.zip'


# Request data
data = requests.get(data_url)
# convert to pandas dataframe
tract2020 = geopandas.read_file(BytesIO(data.content))

In [ ]:
# Spatially join points to 2020 census tracts
interpolated_values = geopandas.sjoin(variables_per_point[["geometry","B01003_001E", "B02001_002E", "B01001_026E","B01003_001M", "B02001_002T", "B01001_026T"]], tract2020, how="right", op='within')

# Assign 0 value to tracts with no points
interpolated_values = interpolated_values.fillna(value=0)

# Propogate error through the summation of points: dh_m = sqrt( sum_{i=1}^n delta_i dg^2) : where delta_i=1 if the it point is in the mth 2020 census tract and delta_i=0 otherwise
interpolated_errors = interpolated_values[["B01003_001M", "B02001_002T", "B01001_026T"]].astype(float).apply(np.square)
interpolated_errors["GEOID"] = interpolated_values["GEOID"]
interpolated_errors = (interpolated_errors.groupby("GEOID").sum()).apply(np.sqrt)

# Sum the values for each 2020 tract
interpolated_values = interpolated_values[["GEOID", "B01003_001E", "B02001_002E", "B01001_026E"]].groupby("GEOID").sum()

# Join values and errors into a completed dataframe
final_df = interpolated_values.join(interpolated_errors, on="GEOID")

# Reset GEOID from index to column
final_df = final_df.reset_index()

In [ ]:
# Pull population data for 2020 Census tracts
# Define request parameters

year = '2020' # Year of interest
datasource = 'acs' # Survey name
subsource = 'acs5' # Subsurvey name
GET = 'B01003_001E,B01003_001M,B02001_002E,B02001_002M,B01001_026E,B01001_026M' # Variables to query
FOR = 'tract:*' # for predicate
IN = 'state:10' # in predicate


# Filepath to your Census API key
keyfile = 'CensusAPIKey.txt'

# Formatted API call
data_url = f'https://api.census.gov/data/{year}/{datasource}/{subsource}?get={GET}&for={FOR}&in={IN}'

# Read Census key into 'api_key'
with open(keyfile) as key:
    api_key = key.read().strip()

# Add key to url
data_url = f'{data_url}&key={api_key}'

# Request data and convert from json
data = requests.get(data_url).json()
# First entry in list is a list of variable names
tract2020_data = pd.DataFrame(data[1:], columns = data[0])

# Add a GEOID column to the data
tract2020_data["GEOID"] = tract2020_data["state"].astype(str) + tract2020_data["county"].astype(str) +tract2020_data["tract"].astype(str)

In [ ]:
final_df.merge(tract2020_data, left_on="GEOID", right_on="GEOID", how="left")

In [ ]:
# Write combined dataframe of 2020 ground truth and estimated values to a csv
final_df.merge(tract2020_data, left_on="GEOID", right_on="GEOID").to_csv("estimates.csv", index=False)